In [ ]:
import os

import numpy as np
import pylab as plt
import pyrap.tables as pt
from daskms.experimental.zarr import xds_from_zarr
from h5parm import DataPack

ionosphere_file = 'sim_dsa2000W_1000m_grid_dawn_30.0_1.5.h5'
sim_base_name = 'W-64chan-20int-iono-only'

[gains] = xds_from_zarr(os.path.join('output', f'{sim_base_name}', 'gains::G'))
sim_gains = np.load(os.path.join('data', f'{sim_base_name}.npy'))
print("Sim Gain Shape", sim_gains.shape)  # time, ant, freq, pol, pol

with pt.table(os.path.join('data', f"{sim_base_name}.ms", "ANTENNA")) as t:
    ant_pos_map = dict(zip(t.getcol('STATION'), t.getcol('POSITION')))
    ant_pos = t.getcol('POSITION')

In [ ]:
gains.params

In [ ]:
with DataPack(os.path.join('output', ionosphere_file), readonly=True) as dp:
    print(dp)
    sim_phase_h5, axes = dp.phase
    sim_phase_h5 = sim_phase_h5[0, 0, :, 0, :] * 180. / np.pi
    sim_tec_h5, _ = dp.tec
    sim_tec_h5 = sim_tec_h5[0, 0, :, :]
plt.imshow(sim_tec_h5[:, :], aspect='auto', cmap='jet')
plt.colorbar()
plt.show()

In [ ]:
freqs = gains.params.param_f.values
times = gains.params.param_t.values
phase = gains.params.values * 180 / np.pi
sim_phase = np.angle(sim_gains) * 180 / np.pi

In [ ]:
plt.plot(sim_phase[0, :, 0, 0, 0])

plt.title("Simulated Phase XX")
plt.xlabel("Ant Index")
plt.ylabel("Phase [deg]")
plt.show()

plt.plot(phase[0, 0, :, 0, 0])
plt.title("Solved Phase XX")
plt.xlabel("Ant Index")
plt.ylabel("Phase [deg]")
plt.show()

In [ ]:
plt.scatter(ant_pos[:, 0], ant_pos[:, 1], c=phase[0, 0, :, 0, 0],
            vmin=-10.,
            vmax=10.,
            cmap='jet')

plt.colorbar(label='Phase [deg]')
plt.xlabel('X [m]')
plt.ylabel('Y [m]')
plt.title("Solved Phase XX")

plt.show()

plt.scatter(ant_pos[:, 0], ant_pos[:, 1], c=sim_phase[0, :, 0, 0, 0], cmap='jet', vmin=-10, vmax=10)
plt.colorbar(label='Phase [deg]')
plt.xlabel('X [m]')
plt.ylabel('Y [m]')
plt.title("Simulated Phase XX")
plt.show()

diff_phase = sim_phase[0, :, 0, 0, 0] - phase[0, 0, :, 0, 0]
plt.scatter(ant_pos[:, 0], ant_pos[:, 1], c=diff_phase, cmap='jet', vmin=-3, vmax=3)
plt.colorbar(label='Phase [deg]')
plt.xlabel('X [m]')
plt.ylabel('Y [m]')
plt.title("Difference from ground truth Phase XX")
plt.show()

bias = np.mean(diff_phase)
stddev = np.std(diff_phase)
print(f"Bias: {bias} deg")
print(f"Sigma: {stddev} deg")

plt.hist(diff_phase, bins='auto')
plt.gca().axvline(bias, c='red', ls='solid')
plt.gca().axvline(bias + stddev, c='red', ls='dashed')
plt.gca().axvline(bias - stddev, c='red', ls='dashed')
plt.xlabel(r'$\Delta$ Phase [deg]')
plt.show()

In [ ]:
print("RMSE:", np.sqrt(np.mean(np.square(phase))))

# all_time_freq RMSE: 3.3249414456896194
# all_freq RMSE: 7.004088534317038
# all_time RMSE: 17.744149825227762

In [ ]:
Nt = 7
Nf = 32

n_all_time_freq = 1
n_all_freq = Nt
n_all_time = Nf
n_1_1 = Nf * Nt

rmse_all_time_freq = 3.32
rmse_all_freq = 7.00
rmse_all_time = 17.74
rmse_1_1 = np.nan

dof = np.asarray([n_all_time_freq, n_all_freq, n_all_time, n_1_1])
rmse = np.asarray([rmse_all_time_freq, rmse_all_freq, rmse_all_time, rmse_1_1])
expected = rmse[0] * np.sqrt(dof / dof[0])

plt.scatter(dof, rmse)
plt.plot(dof, expected)
plt.show()